In [3]:
!pip install ultralytics pycocotools -q

In [4]:
import os
import csv
import json
import yaml
import random
import shutil
import time
from datetime import datetime
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm as TQDM

from pycocotools.coco import COCO
import numpy as np
import pandas as pd

import torch

from ultralytics import YOLO
import ultralytics
from ultralytics.utils.files import increment_path
from ultralytics.data.converter import coco91_to_coco80_class

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
SEED = 42
EPOCHS = 50
BATCH_SIZE = 128
IMAGE_SIZE = 512
PROJECT_NAME = "yolo8n_pt_512_coco_skiped_crowd_3_85"
BASE_MODEL = "yolov8n.pt"
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
PATIENCE = 20

INPUT_DATASET_ROOT = "/kaggle/input/coco-train/coco"
DATASET_ROOT = "/kaggle/working/dataset"
TRAINING_ROOT = "/kaggle/working/training"

PATHS = {
    'input_root': INPUT_DATASET_ROOT,
    'dataset_root': DATASET_ROOT,
    'training_root': TRAINING_ROOT,
    'train_images': os.path.join(INPUT_DATASET_ROOT, "train"),
    'val_images': os.path.join(INPUT_DATASET_ROOT, "val"),
    'train_annotations': os.path.join(INPUT_DATASET_ROOT, "annotations", "instances_train.json"),
    'val_annotations': os.path.join(INPUT_DATASET_ROOT, "annotations", "instances_val.json"),
    'yaml_file': os.path.join(DATASET_ROOT, "yolo_main.yaml"),
    'trained_model': os.path.join(TRAINING_ROOT, PROJECT_NAME, "weights", "best.pt"),
}

BBOX_AREA_TRESHOLD_MIN = 0.003
BBOX_AREA_TRESHOLD_MAX = 0.85
USE_SIMLINKS = True
SKIP_CROWD_IMAGES = True

print("The configuration is set:")
print(f"  - EPOCHS: {EPOCHS}")
print(f"  - BATCH_SIZE: {BATCH_SIZE}")
print(f"  - IMAGE_SIZE: {IMAGE_SIZE}")
print(f"  - DEVICE: {DEVICE}")
print(f"  - BASE_MODEL: {BASE_MODEL}")

The configuration is set:
  - EPOCHS: 50
  - BATCH_SIZE: 128
  - IMAGE_SIZE: 512
  - DEVICE: cuda
  - BASE_MODEL: yolov8n.pt


In [6]:
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

ultralytics.checks()

Ultralytics 8.3.167 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6411.3/8062.4 GB disk)


In [7]:
def custom_convert_coco(
    labels_dir: str = "../coco/annotations/",
    save_dir: str = "coco_converted/",
    cls91to80: bool = True,
    bbox_area_threshold_min: float = 0,
    bbox_area_threshold_max: float = 1,
    use_symlinks: bool = False,
    skip_crowd_images: bool = False,
):
    """
    Convert COCO-style JSON annotations into YOLO format, filtering by bbox area
    and optionally dropping images marked with iscrowd==1.

    Parameters
    ----------
    labels_dir : str
        Path to COCO 'annotations' folder.
    save_dir : str
        Output root for 'images' and 'labels' subfolders.
    cls91to80 : bool
        Map COCO91 IDs to COCO80 if True.
    bbox_area_threshold_min : float
        Min relative bbox area (w*h / img_area) to keep an annotation.
    bbox_area_threshold_max : float
        Max relative bbox area to keep an annotation; if exceeded, drop image.
    use_symlinks : bool
        Create symlinks to the original images instead of copying.
    skip_crowd_images : bool
        If True, drop any image that has at least one annotation with iscrowd==1.
    """
    save_dir = Path(save_dir).expanduser().resolve()
    (save_dir / "labels").mkdir(parents=True, exist_ok=True)
    (save_dir / "images").mkdir(parents=True, exist_ok=True)

    coco80 = coco91_to_coco80_class()
    all_categories = {}

    n_images_saved = 0
    n_images_skipped_big_bbox = 0
    n_images_skipped_crowd = 0
    n_annotations_removed_small = 0
    n_annotations_kept = 0
    kept_train, kept_val = [], []

    for json_file in sorted(Path(labels_dir).glob("*.json")):
        split = "train" if "train" in json_file.stem.lower() else "val"
        data = json.loads(json_file.read_text(encoding="utf-8"))

        # build category lookup
        for cat in data.get("categories", []):
            all_categories[cat["id"]] = cat["name"]

        # index images and annotations by image_id
        images = {str(img['id']): img for img in data['images']}
        anns_by_image = defaultdict(list)
        crowd_by_image = defaultdict(bool)
        for ann in data['annotations']:
            img_id = str(ann['image_id'])
            anns_by_image[img_id].append(ann)
            if ann.get('iscrowd', 0) == 1:
                crowd_by_image[img_id] = True

        # process each image
        for img_id, img in TQDM(images.items(), desc=f"Processing {json_file.name}"):
            # skip if crowd flag and we want to drop crowd images
            if skip_crowd_images and crowd_by_image.get(img_id, False):
                n_images_skipped_crowd += 1
                continue

            w, h = img['width'], img['height']
            img_area = w * h
            fname = img['file_name']
            anns = anns_by_image.get(img_id, [])

            valid_anns = []
            skip_image = False

            # filter annotations by relative bbox area
            for ann in anns:
                bw, bh = ann['bbox'][2], ann['bbox'][3]
                rel = (bw * bh) / img_area
                if rel < bbox_area_threshold_min:
                    n_annotations_removed_small += 1
                    continue
                if rel > bbox_area_threshold_max:
                    skip_image = True
                    break
                valid_anns.append(ann)

            if skip_image:
                n_images_skipped_big_bbox += 1
                continue

            # record kept filename
            (kept_train if split == "train" else kept_val).append(fname)

            # prepare output directories
            img_dir = save_dir / "images" / split
            lbl_dir = save_dir / "labels" / split
            img_dir.mkdir(parents=True, exist_ok=True)
            lbl_dir.mkdir(parents=True, exist_ok=True)

            # copy or symlink the image
            src = Path(labels_dir).parent / split / fname
            dst = img_dir / fname
            if use_symlinks:
                if not dst.exists():
                    os.symlink(src, dst)
            else:
                if src.exists():
                    shutil.copy(src, dst)
                else:
                    print(f"Image not found: {src}, skipping.")

            # write YOLO-format label file
            out_f = lbl_dir / Path(fname).with_suffix(".txt")
            with open(out_f, "w", encoding="utf-8") as out:
                for ann in valid_anns:
                    box = np.array(ann['bbox'], dtype=float)
                    # convert from [x, y, w, h] to [x_center, y_center, w, h] normalized
                    box[:2] += box[2:] / 2
                    box[[0, 2]] /= w
                    box[[1, 3]] /= h
                    cls = (
                        coco80[ann['category_id'] - 1]
                        if cls91to80
                        else ann['category_id'] - 1
                    )
                    line = [cls] + box.tolist()
                    out.write(("%g " * len(line)).rstrip() % tuple(line) + "\n")

            n_images_saved += 1
            n_annotations_kept += len(valid_anns)

    # report stats
    print(f"Saved images: {n_images_saved}")
    print(f"Skipped (large bbox): {n_images_skipped_big_bbox}")
    print(f"Skipped (crowd images): {n_images_skipped_crowd}")
    print(f"Removed small anns: {n_annotations_removed_small}")
    print(f"Kept anns: {n_annotations_kept}")
    print(f"Train count: {len(kept_train)}, Val count: {len(kept_val)}")

    yaml_cats = {i - 1: n for i, n in all_categories.items()}
    return (
        save_dir,
        kept_train,
        kept_val,
        yaml_cats,
        n_images_saved,
        n_images_skipped_big_bbox,
        n_images_skipped_crowd,
        n_annotations_removed_small,
        n_annotations_kept,
    )


Annotations /kaggle/input/coco-train/coco/annotations/instances_train.json: 100%|██████████| 19500/19500 [00:02<00:00, 6872.11it/s]
Annotations /kaggle/input/coco-train/coco/annotations/instances_val.json: 100%|██████████| 5789/5789 [00:00<00:00, 6883.61it/s]

COCO data converted successfully.
Results saved to /kaggle/working/dataset


In [ ]:
save_dir, kept_tr, kept_val, yaml_cats, n_images_saved, n_images_skipped_big_bbox, n_images_skipped_crowd, n_annotations_removed_small, n_annotations_kept = custom_convert_coco(
    labels_dir=os.path.join(PATHS['input_root'], "annotations"),
    save_dir=PATHS['dataset_root'],
    bbox_area_threshold_min=BBOX_AREA_TRESHOLD_MIN,
    bbox_area_threshold_max=BBOX_AREA_TRESHOLD_MAX,
    use_symlinks=USE_SIMLINKS,
    skip_crowd_images=SKIP_CROWD_IMAGES
)

In [ ]:
# Creating a YAML configuration for YOLO
data_yaml = {
    "path": PATHS['dataset_root'],
    "train": "images/train",
    "val": "images/val",
    "names": yaml_cats
}

with open(PATHS['yaml_file'], "w") as f:
    yaml.dump(data_yaml, f, default_flow_style=False, allow_unicode=True)

print(f"YAML configuration is saved: {PATHS['yaml_file']}")
print("YAML configuration:")
print(yaml.dump(data_yaml, default_flow_style=False, allow_unicode=True))

In [ ]:
print(f"Model initialization: {BASE_MODEL}")
model = YOLO(BASE_MODEL)
print(f"Model initialized. Device in use: {DEVICE}")

In [ ]:
print(f"Training configuration:")
print(f"  - Epochs: {EPOCHS}")
print(f"  - Batch size: {BATCH_SIZE}")
print(f"  - Image size: {IMAGE_SIZE}")
print(f"  - Device: {DEVICE}")
print(f"  - Patience: {PATIENCE}")

training_results = model.train(
    data=PATHS['yaml_file'],
    epochs=EPOCHS,
    batch=BATCH_SIZE,
    imgsz=IMAGE_SIZE,
    project=PATHS['training_root'],
    name=PROJECT_NAME,
    device=DEVICE,
    seed=SEED,
    patience=PATIENCE,
)

trained_model_path = PATHS['trained_model']
print(f"Model saved: {trained_model_path}")

In [ ]:
def create_training_readme():
    """Create a comprehensive README for the training results with detailed information"""

    # Get training results path
    results_path = os.path.join(PATHS['training_root'], PROJECT_NAME)
    readme_path = os.path.join(results_path, 'README.md')

    # Read results if available
    results_csv = os.path.join(results_path, 'results.csv')

    # Analyze dataset for detailed class statistics
    def analyze_dataset_classes():
        """Analyze the converted dataset to get detailed class statistics"""
        class_stats = defaultdict(lambda: {'train': 0, 'val': 0, 'total': 0})
        train_labels_dir = os.path.join(PATHS['dataset_root'], 'labels', 'train')
        val_labels_dir = os.path.join(PATHS['dataset_root'], 'labels', 'val')

        total_train_annotations = 0
        total_val_annotations = 0

        # Analyze training labels
        if os.path.exists(train_labels_dir):
            for label_file in Path(train_labels_dir).glob('*.txt'):
                with open(label_file, 'r') as f:
                    for line in f:
                        if line.strip():
                            class_id = int(line.strip().split()[0])
                            class_stats[class_id]['train'] += 1
                            total_train_annotations += 1

        # Analyze validation labels
        if os.path.exists(val_labels_dir):
            for label_file in Path(val_labels_dir).glob('*.txt'):
                with open(label_file, 'r') as f:
                    for line in f:
                        if line.strip():
                            class_id = int(line.strip().split()[0])
                            class_stats[class_id]['val'] += 1
                            total_val_annotations += 1

        # Calculate totals
        for class_id in class_stats:
            class_stats[class_id]['total'] = class_stats[class_id]['train'] + class_stats[class_id]['val']

        return dict(class_stats), total_train_annotations, total_val_annotations

    # Get detailed class statistics
    class_stats, total_train_anns, total_val_anns = analyze_dataset_classes()

    # Calculate additional dataset metrics
    train_val_ratio = len(kept_tr) / (len(kept_tr) + len(kept_val)) if (len(kept_tr) + len(kept_val)) > 0 else 0
    avg_anns_per_train_img = total_train_anns / len(kept_tr) if len(kept_tr) > 0 else 0
    avg_anns_per_val_img = total_val_anns / len(kept_val) if len(kept_val) > 0 else 0

    readme_content = f"""# YOLO Training Results - {PROJECT_NAME}

## 📋 Training Overview
**Model:** {BASE_MODEL}
**Project:** {PROJECT_NAME}
**Training Date:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
**Seed:** {SEED} (for reproducibility)
**Training Duration:** {'Completed' if os.path.exists(results_csv) else 'In Progress'}

## ⚙️ Training Configuration

### Hyperparameters
| Parameter | Value |
|-----------|-------|
| **Epochs** | {EPOCHS} |
| **Batch Size** | {BATCH_SIZE} |
| **Image Size** | {IMAGE_SIZE}×{IMAGE_SIZE} |
| **Device** | {DEVICE} |
| **Patience** | {PATIENCE} |
| **Random Seed** | {SEED} |
| **Learning Rate** | Default YOLO |
| **Optimizer** | AdamW |
| **Weight Decay** | Default YOLO |

### Environment Information
| Component | Version/Info |
|-----------|--------------|
| **Python** | 3.11.11 |
| **PyTorch** | {torch.__version__} |
| **Ultralytics** | {ultralytics.__version__} |
| **CUDA Available** | {torch.cuda.is_available()} |
| **GPU Memory** | {'Available' if torch.cuda.is_available() else 'N/A'} |
| **Device Used** | {DEVICE} |
| **Platform** | Kaggle Notebook |

## 📊 Dataset Information

### Dataset Overview
| Metric | Value | Percentage |
|--------|-------|------------|
| **Dataset Source** | `{INPUT_DATASET_ROOT}` | - |
| **Training Images** | {len(kept_tr):,} | {len(kept_tr)/(len(kept_tr) + len(kept_val))*100:.1f}% |
| **Validation Images** | {len(kept_val):,} | {len(kept_val)/(len(kept_tr) + len(kept_val))*100:.1f}% |
| **Total Images** | {len(kept_tr) + len(kept_val):,} | 100% |
| **Training Annotations** | {total_train_anns:,} | {total_train_anns/(total_train_anns + total_val_anns)*100:.1f}% |
| **Validation Annotations** | {total_val_anns:,} | {total_val_anns/(total_train_anns + total_val_anns)*100:.1f}% |
| **Total Annotations** | {total_train_anns + total_val_anns:,} | 100% |

### Dataset Quality Metrics
| Metric | Value |
|--------|-------|
| **Avg Annotations per Train Image** | {avg_anns_per_train_img:.2f} |
| **Avg Annotations per Val Image** | {avg_anns_per_val_img:.2f} |
| **Overall Avg Annotations per Image** | {(total_train_anns + total_val_anns)/(len(kept_tr) + len(kept_val)):.2f} |
| **Dataset Balance (Train/Val)** | {train_val_ratio:.1%} / {1-train_val_ratio:.1%} |
| **Annotation Density** | {(total_train_anns + total_val_anns)/(len(kept_tr) + len(kept_val)):.2f} objects per image |

### Dataset Processing Pipeline
| Processing Step | Count | Percentage | Notes |
|----------------|-------|------------|-------|
| **Original Images** | {n_images_saved + n_images_skipped_big_bbox + n_images_skipped_crowd:,} | 100% | Total in source dataset |
| **Images Saved** | {n_images_saved:,} | {n_images_saved/(n_images_saved + n_images_skipped_big_bbox + n_images_skipped_crowd)*100:.1f}% | Successfully processed |
| **Images Skipped (Large BBox)** | {n_images_skipped_big_bbox:,} | {n_images_skipped_big_bbox/(n_images_saved + n_images_skipped_big_bbox + n_images_skipped_crowd)*100:.1f}% | BBox > {BBOX_AREA_TRESHOLD_MAX} of image area |
| **Images Skipped (Crowd)** | {n_images_skipped_crowd:,} | {n_images_skipped_crowd/(n_images_saved + n_images_skipped_big_bbox + n_images_skipped_crowd)*100:.1f}% | Contains crowd annotations |
| **Annotations Removed (Small)** | {n_annotations_removed_small:,} | - | BBox < {BBOX_AREA_TRESHOLD_MIN} of image area |
| **Annotations Kept** | {n_annotations_kept:,} | - | Final annotation count |

### Data Filtering Parameters
| Parameter | Value | Description |
|-----------|-------|-------------|
| **Min BBox Area Threshold** | {BBOX_AREA_TRESHOLD_MIN} | Minimum relative bbox area (w×h / image_area) |
| **Max BBox Area Threshold** | {BBOX_AREA_TRESHOLD_MAX} | Maximum relative bbox area (drops entire image) |
| **Skip Crowd Images** | {SKIP_CROWD_IMAGES} | Remove images with crowd annotations |
| **Use Symlinks** | {USE_SIMLINKS} | Create symlinks instead of copying images |

## 🏷️ Class Distribution Analysis

### Class Overview
**Total Number of Classes:** {len(yaml_cats)}
**Most Frequent Class:** {max(class_stats.keys(), key=lambda x: class_stats[x]['total']) if class_stats else 'N/A'} ({yaml_cats.get(max(class_stats.keys(), key=lambda x: class_stats[x]['total']), 'N/A') if class_stats else 'N/A'}) - {max(class_stats.values(), key=lambda x: x['total'])['total'] if class_stats else 0:,} annotations
**Least Frequent Class:** {min(class_stats.keys(), key=lambda x: class_stats[x]['total']) if class_stats else 'N/A'} ({yaml_cats.get(min(class_stats.keys(), key=lambda x: class_stats[x]['total']), 'N/A') if class_stats else 'N/A'}) - {min(class_stats.values(), key=lambda x: x['total'])['total'] if class_stats else 0:,} annotations"""

    # Add detailed class statistics table
    if class_stats:
        readme_content += f"""

### 📊 Detailed Class Statistics
| Class ID | Class Name | Train Annotations | Val Annotations | Total Annotations | Train % | Val % | Total % |
|----------|------------|-------------------|-----------------|-------------------|---------|-------|---------|"""

        # Sort classes by total annotations (descending)
        sorted_classes = sorted(class_stats.items(), key=lambda x: x[1]['total'], reverse=True)

        for class_id, stats in sorted_classes:
            class_name = yaml_cats.get(class_id, f'Class_{class_id}')
            train_count = stats['train']
            val_count = stats['val']
            total_count = stats['total']
            train_pct = (train_count / total_train_anns * 100) if total_train_anns > 0 else 0
            val_pct = (val_count / total_val_anns * 100) if total_val_anns > 0 else 0
            total_pct = (total_count / (total_train_anns + total_val_anns) * 100) if (total_train_anns + total_val_anns) > 0 else 0

            readme_content += f"\n| {class_id} | {class_name} | {train_count:,} | {val_count:,} | {total_count:,} | {train_pct:.2f}% | {val_pct:.2f}% | {total_pct:.2f}% |"

        # Add class distribution summary
        total_classes_with_data = len([c for c in class_stats.values() if c['total'] > 0])
        classes_with_low_samples = len([c for c in class_stats.values() if c['total'] < 100])
        classes_with_high_samples = len([c for c in class_stats.values() if c['total'] > 1000])

        readme_content += f"""

### 📈 Class Distribution Summary
| Metric | Value |
|--------|-------|
| **Classes with Annotations** | {total_classes_with_data} / {len(yaml_cats)} |
| **Classes with <100 Annotations** | {classes_with_low_samples} ({classes_with_low_samples/len(yaml_cats)*100:.1f}%) |
| **Classes with >1000 Annotations** | {classes_with_high_samples} ({classes_with_high_samples/len(yaml_cats)*100:.1f}%) |
| **Class Imbalance Ratio** | {max(class_stats.values(), key=lambda x: x['total'])['total'] / max(1, min(class_stats.values(), key=lambda x: x['total'])['total']) if class_stats else 0:.1f}:1 |
| **Average Annotations per Class** | {(total_train_anns + total_val_anns) / len(yaml_cats):.1f} |
| **Median Annotations per Class** | {sorted([c['total'] for c in class_stats.values()])[len(class_stats)//2] if class_stats else 0} |"""

    # Add complete class mapping
    readme_content += f"""

### 🗂️ Complete Class Mapping
```yaml
# COCO80 Class Names (ID: Name)
{yaml.dump(yaml_cats, default_flow_style=False, allow_unicode=True, width=float('inf'))}```"""

    readme_content += f"""

## 📈 Training Results"""

    # Add detailed training metrics if results.csv exists
    if os.path.exists(results_csv):
        try:
            df = pd.read_csv(results_csv)
            last_epoch = df.iloc[-1]
            best_epoch_idx = df['metrics/mAP50(B)'].idxmax() if 'metrics/mAP50(B)' in df.columns else -1
            best_epoch = df.iloc[best_epoch_idx] if best_epoch_idx != -1 else last_epoch

            # Calculate training efficiency metrics
            training_duration = len(df)
            epochs_to_best = int(best_epoch['epoch']) + 1 if best_epoch_idx != -1 else training_duration
            training_efficiency = (epochs_to_best / EPOCHS) * 100

            readme_content += f"""

### 🎯 Training Summary
| Metric | Value |
|--------|-------|
| **Total Epochs Completed** | {training_duration} / {EPOCHS} |
| **Best Performance at Epoch** | {int(best_epoch['epoch']) + 1} |
| **Training Efficiency** | {training_efficiency:.1f}% (reached best at {training_efficiency:.1f}% of max epochs) |
| **Early Stopping Triggered** | {'Yes' if training_duration < EPOCHS else 'No'} |
| **Final Learning Rate** | {last_epoch.get('lr/pg0', 'N/A')} |

### 🏆 Best Performance Metrics (Epoch {int(best_epoch['epoch']) + 1})
| Metric | Value | Description |
|--------|-------|-------------|
| **mAP@0.5** | {best_epoch.get('metrics/mAP50(B)', 'N/A'):.4f} | Mean Average Precision at IoU=0.5 |
| **mAP@0.5:0.95** | {best_epoch.get('metrics/mAP50-95(B)', 'N/A'):.4f} | Mean Average Precision averaged over IoU 0.5-0.95 |
| **Precision** | {best_epoch.get('metrics/precision(B)', 'N/A'):.4f} | True Positives / (True Positives + False Positives) |
| **Recall** | {best_epoch.get('metrics/recall(B)', 'N/A'):.4f} | True Positives / (True Positives + False Negatives) |
| **F1-Score** | {2 * best_epoch.get('metrics/precision(B)', 0) * best_epoch.get('metrics/recall(B)', 0) / (best_epoch.get('metrics/precision(B)', 0) + best_epoch.get('metrics/recall(B)', 0) + 1e-16):.4f} | Harmonic mean of Precision and Recall |

### 📊 Final Training Metrics (Epoch {int(last_epoch['epoch']) + 1})

#### Loss Metrics
| Loss Type | Train | Validation | Description |
|-----------|-------|------------|-------------|
| **Box Loss** | {last_epoch.get('train/box_loss', 'N/A'):.6f} | {last_epoch.get('val/box_loss', 'N/A'):.6f} | Bounding box regression loss |
| **Class Loss** | {last_epoch.get('train/cls_loss', 'N/A'):.6f} | {last_epoch.get('val/cls_loss', 'N/A'):.6f} | Classification loss |
| **DFL Loss** | {last_epoch.get('train/dfl_loss', 'N/A'):.6f} | {last_epoch.get('val/dfl_loss', 'N/A'):.6f} | Distribution Focal Loss |

#### Performance Metrics
| Metric | Value | Benchmark |
|--------|-------|-----------|
| **Precision** | {last_epoch.get('metrics/precision(B)', 'N/A'):.4f} | >0.7 Good, >0.8 Excellent |
| **Recall** | {last_epoch.get('metrics/recall(B)', 'N/A'):.4f} | >0.7 Good, >0.8 Excellent |
| **mAP@0.5** | {last_epoch.get('metrics/mAP50(B)', 'N/A'):.4f} | >0.5 Good, >0.7 Excellent |
| **mAP@0.5:0.95** | {last_epoch.get('metrics/mAP50-95(B)', 'N/A'):.4f} | >0.3 Good, >0.5 Excellent |
| **F1-Score** | {2 * last_epoch.get('metrics/precision(B)', 0) * last_epoch.get('metrics/recall(B)', 0) / (last_epoch.get('metrics/precision(B)', 0) + last_epoch.get('metrics/recall(B)', 0) + 1e-16):.4f} | Balanced precision-recall metric |

### 📊 Training Progress Analysis
| Metric | First Epoch | Last Epoch | Best Value | Improvement | Epoch of Best |
|--------|-------------|------------|------------|-------------|---------------|"""

            first_epoch = df.iloc[0]

            # Calculate improvements for key metrics
            metrics_to_track = [
                ('mAP@0.5', 'metrics/mAP50(B)', False),
                ('mAP@0.5:0.95', 'metrics/mAP50-95(B)', False),
                ('Precision', 'metrics/precision(B)', False),
                ('Recall', 'metrics/recall(B)', False),
                ('Train Box Loss', 'train/box_loss', True),
                ('Train Class Loss', 'train/cls_loss', True),
                ('Val Box Loss', 'val/box_loss', True),
                ('Val Class Loss', 'val/cls_loss', True)
            ]

            for metric_name, metric_key, is_loss in metrics_to_track:
                if metric_key in df.columns:
                    first_val = first_epoch.get(metric_key, 0)
                    last_val = last_epoch.get(metric_key, 0)
                    if is_loss:
                        best_val = df[metric_key].min()
                        best_epoch_num = df[metric_key].idxmin() + 1
                        improvement = ((first_val - last_val) / first_val * 100) if first_val != 0 else 0
                        improvement_sign = "↓" if improvement > 0 else "↑"
                    else:
                        best_val = df[metric_key].max()
                        best_epoch_num = df[metric_key].idxmax() + 1
                        improvement = ((last_val - first_val) / first_val * 100) if first_val != 0 else 0
                        improvement_sign = "↑" if improvement > 0 else "↓"

                    readme_content += f"\n| **{metric_name}** | {first_val:.4f} | {last_val:.4f} | {best_val:.4f} | {improvement_sign}{abs(improvement):.1f}% | {best_epoch_num} |"

            # Add learning rate and other training dynamics
            if 'lr/pg0' in df.columns:
                readme_content += f"""

### 📉 Training Dynamics
| Metric | Initial | Final | Description |
|--------|---------|-------|-------------|
| **Learning Rate (pg0)** | {df['lr/pg0'].iloc[0]:.6f} | {df['lr/pg0'].iloc[-1]:.6f} | Primary parameter group learning rate |"""

                if 'lr/pg1' in df.columns:
                    readme_content += f"\n| **Learning Rate (pg1)** | {df['lr/pg1'].iloc[0]:.6f} | {df['lr/pg1'].iloc[-1]:.6f} | Secondary parameter group learning rate |"
                if 'lr/pg2' in df.columns:
                    readme_content += f"\n| **Learning Rate (pg2)** | {df['lr/pg2'].iloc[0]:.6f} | {df['lr/pg2'].iloc[-1]:.6f} | Tertiary parameter group learning rate |"

            # Add epoch-by-epoch breakdown for key metrics
            readme_content += f"""

### 📋 Detailed Training Log (Key Epochs)
| Epoch | Box Loss | Cls Loss | Val Loss | mAP@0.5 | mAP@0.5:0.95 | Precision | Recall | LR |
|-------|----------|----------|----------|---------|--------------|-----------|--------|-----|"""

            # Show strategic epochs: first, every 10th, best performance, and last
            epochs_to_show = [0]  # First epoch
            epochs_to_show.extend(range(9, len(df), 10))  # Every 10th epoch
            if best_epoch_idx not in epochs_to_show and best_epoch_idx != -1:
                epochs_to_show.append(best_epoch_idx)  # Best epoch
            if len(df) - 1 not in epochs_to_show:
                epochs_to_show.append(len(df) - 1)  # Last epoch

            epochs_to_show = sorted(set(epochs_to_show))

            for i in epochs_to_show:
                row = df.iloc[i]
                epoch_num = int(row['epoch']) + 1
                box_loss = row.get('train/box_loss', 0)
                cls_loss = row.get('train/cls_loss', 0)
                val_loss = row.get('val/box_loss', 0) + row.get('val/cls_loss', 0)
                map50 = row.get('metrics/mAP50(B)', 0)
                map50_95 = row.get('metrics/mAP50-95(B)', 0)
                precision = row.get('metrics/precision(B)', 0)
                recall = row.get('metrics/recall(B)', 0)
                lr = row.get('lr/pg0', 0)

                # Highlight best epoch
                marker = " 🏆" if i == best_epoch_idx else ""
                readme_content += f"\n| {epoch_num}{marker} | {box_loss:.4f} | {cls_loss:.4f} | {val_loss:.4f} | {map50:.4f} | {map50_95:.4f} | {precision:.4f} | {recall:.4f} | {lr:.6f} |"

        except Exception as e:
            readme_content += f"\n\n*⚠️ Error loading detailed training results: {str(e)}*\n"

    # Add comprehensive model and deployment information
    readme_content += f"""


## 📊 Additional Metrics & Analysis

### Training Efficiency
- **Compute Hours**: ~{EPOCHS * BATCH_SIZE * (len(kept_tr) + len(kept_val)) / (3600 * 1000):.2f} GPU hours estimated
- **Samples per Second**: ~{BATCH_SIZE * len(kept_tr) / (EPOCHS * 60):.0f} during training
- **Memory Usage**: ~{BATCH_SIZE * 3 * IMAGE_SIZE * IMAGE_SIZE * 4 / (1024**3):.2f} GB GPU memory (estimated)

### Data Quality Assessment
- **Annotation Quality**: Filtered for size and crowd constraints
- **Class Balance**: {len([c for c in class_stats.values() if c['total'] > (total_train_anns + total_val_anns) / (len(yaml_cats) * 2)])} classes above average
- **Dataset Completeness**: {len(kept_tr) + len(kept_val):,} images from {n_images_saved + n_images_skipped_big_bbox + n_images_skipped_crowd:,} original


*Generated automatically on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*
*Training: {len(kept_tr):,} images | Validation: {len(kept_val):,} images | Classes: {len(yaml_cats)}*
*Device: {DEVICE} | Epochs: {EPOCHS} | Batch Size: {BATCH_SIZE} | Image Size: {IMAGE_SIZE}×{IMAGE_SIZE}*
*Total Annotations: {total_train_anns + total_val_anns:,} | Avg per Image: {(total_train_anns + total_val_anns)/(len(kept_tr) + len(kept_val)):.1f}*
"""

    # Write README
    with open(readme_path, 'w', encoding='utf-8') as f:
        f.write(readme_content)

    print(f"✅ Enhanced README created: {readme_path}")

    # Create detailed summary JSON with all available information
    summary_data = {
        "project_info": {
            "name": PROJECT_NAME,
            "base_model": BASE_MODEL,
            "timestamp": datetime.now().isoformat(),
            "seed": SEED,
            "training_completed": os.path.exists(results_csv)
        },
        "hyperparameters": {
            "epochs": EPOCHS,
            "batch_size": BATCH_SIZE,
            "image_size": IMAGE_SIZE,
            "device": DEVICE,
            "patience": PATIENCE,
            "bbox_area_threshold_min": BBOX_AREA_TRESHOLD_MIN,
            "bbox_area_threshold_max": BBOX_AREA_TRESHOLD_MAX,
            "skip_crowd_images": SKIP_CROWD_IMAGES,
            "use_symlinks": USE_SIMLINKS
        },
        "dataset": {
            "source": INPUT_DATASET_ROOT,
            "train_images": len(kept_tr),
            "val_images": len(kept_val),
            "total_images": len(kept_tr) + len(kept_val),
            "train_annotations": total_train_anns,
            "val_annotations": total_val_anns,
            "total_annotations": total_train_anns + total_val_anns,
            "train_val_ratio": f"{len(kept_tr)/(len(kept_tr) + len(kept_val))*100:.1f}%/{len(kept_val)/(len(kept_tr) + len(kept_val))*100:.1f}%",
            "avg_annotations_per_train_image": avg_anns_per_train_img,
            "avg_annotations_per_val_image": avg_anns_per_val_img,
            "avg_annotations_per_image": (total_train_anns + total_val_anns)/(len(kept_tr) + len(kept_val)),
            "num_classes": len(yaml_cats),
            "class_names": yaml_cats,
            "class_statistics": class_stats
        },
        "data_processing": {
            "images_saved": n_images_saved,
            "images_skipped_big_bbox": n_images_skipped_big_bbox,
            "images_skipped_crowd": n_images_skipped_crowd,
            "annotations_removed_small": n_annotations_removed_small,
            "annotations_kept": n_annotations_kept,
            "processing_efficiency": n_images_saved/(n_images_saved + n_images_skipped_big_bbox + n_images_skipped_crowd) if (n_images_saved + n_images_skipped_big_bbox + n_images_skipped_crowd) > 0 else 0
        },
        "class_analysis": {
            "total_classes_with_data": len([c for c in class_stats.values() if c['total'] > 0]) if class_stats else 0,
            "classes_with_low_samples": len([c for c in class_stats.values() if c['total'] < 100]) if class_stats else 0,
            "classes_with_high_samples": len([c for c in class_stats.values() if c['total'] > 1000]) if class_stats else 0,
            "most_frequent_class": {
                "id": max(class_stats.keys(), key=lambda x: class_stats[x]['total']) if class_stats else None,
                "name": yaml_cats.get(max(class_stats.keys(), key=lambda x: class_stats[x]['total']), 'N/A') if class_stats else 'N/A',
                "count": max(class_stats.values(), key=lambda x: x['total'])['total'] if class_stats else 0
            },
            "least_frequent_class": {
                "id": min(class_stats.keys(), key=lambda x: class_stats[x]['total']) if class_stats else None,
                "name": yaml_cats.get(min(class_stats.keys(), key=lambda x: class_stats[x]['total']), 'N/A') if class_stats else 'N/A',
                "count": min(class_stats.values(), key=lambda x: x['total'])['total'] if class_stats else 0
            },
            "class_imbalance_ratio": max(class_stats.values(), key=lambda x: x['total'])['total'] / max(1, min(class_stats.values(), key=lambda x: x['total'])['total']) if class_stats else 0,
            "median_annotations_per_class": sorted([c['total'] for c in class_stats.values()])[len(class_stats)//2] if class_stats else 0
        },
        "environment": {
            "python_version": "3.11.11",
            "pytorch_version": torch.__version__,
            "ultralytics_version": ultralytics.__version__,
            "cuda_available": torch.cuda.is_available(),
            "platform": "Kaggle Notebook"
        },
        "paths": {
            "dataset_root": PATHS['dataset_root'],
            "training_root": PATHS['training_root'],
            "yaml_file": PATHS['yaml_file'],
            "trained_model": PATHS['trained_model']
        },
        "estimated_metrics": {
            "estimated_gpu_hours": EPOCHS * BATCH_SIZE * (len(kept_tr) + len(kept_val)) / (3600 * 1000),
            "estimated_samples_per_second": BATCH_SIZE * len(kept_tr) / (EPOCHS * 60),
            "estimated_gpu_memory_gb": BATCH_SIZE * 3 * IMAGE_SIZE * IMAGE_SIZE * 4 / (1024**3)
        }
    }

    # Add training results to summary if available
    if os.path.exists(results_csv):
        try:
            df = pd.read_csv(results_csv)
            last_epoch = df.iloc[-1]
            best_epoch_idx = df['metrics/mAP50(B)'].idxmax() if 'metrics/mAP50(B)' in df.columns else -1
            best_epoch = df.iloc[best_epoch_idx] if best_epoch_idx != -1 else last_epoch
            first_epoch = df.iloc[0]

            # Calculate training efficiency
            training_duration = len(df)
            epochs_to_best = int(best_epoch['epoch']) + 1 if best_epoch_idx != -1 else training_duration
            training_efficiency = (epochs_to_best / EPOCHS) * 100

            summary_data["training_results"] = {
                "total_epochs_trained": training_duration,
                "max_epochs": EPOCHS,
                "epochs_to_best_performance": epochs_to_best,
                "training_efficiency_percent": training_efficiency,
                "early_stopping_triggered": training_duration < EPOCHS,
                "final_metrics": {
                    "epoch": int(last_epoch['epoch']),
                    "train_box_loss": float(last_epoch.get('train/box_loss', 0)),
                    "train_cls_loss": float(last_epoch.get('train/cls_loss', 0)),
                    "train_dfl_loss": float(last_epoch.get('train/dfl_loss', 0)),
                    "val_box_loss": float(last_epoch.get('val/box_loss', 0)),
                    "val_cls_loss": float(last_epoch.get('val/cls_loss', 0)),
                    "val_dfl_loss": float(last_epoch.get('val/dfl_loss', 0)),
                    "precision": float(last_epoch.get('metrics/precision(B)', 0)),
                    "recall": float(last_epoch.get('metrics/recall(B)', 0)),
                    "map50": float(last_epoch.get('metrics/mAP50(B)', 0)),
                    "map50_95": float(last_epoch.get('metrics/mAP50-95(B)', 0)),
                    "f1_score": 2 * last_epoch.get('metrics/precision(B)', 0) * last_epoch.get('metrics/recall(B)', 0) / (last_epoch.get('metrics/precision(B)', 0) + last_epoch.get('metrics/recall(B)', 0) + 1e-16),
                    "learning_rate": float(last_epoch.get('lr/pg0', 0))
                },
                "best_metrics": {
                    "epoch": int(best_epoch['epoch']),
                    "best_map50": float(best_epoch.get('metrics/mAP50(B)', 0)),
                    "best_map50_95": float(best_epoch.get('metrics/mAP50-95(B)', 0)),
                    "precision_at_best": float(best_epoch.get('metrics/precision(B)', 0)),
                    "recall_at_best": float(best_epoch.get('metrics/recall(B)', 0)),
                    "f1_score_at_best": 2 * best_epoch.get('metrics/precision(B)', 0) * best_epoch.get('metrics/recall(B)', 0) / (best_epoch.get('metrics/precision(B)', 0) + best_epoch.get('metrics/recall(B)', 0) + 1e-16)
                },
                "training_progress": {
                    "initial_map50": float(first_epoch.get('metrics/mAP50(B)', 0)),
                    "final_map50": float(last_epoch.get('metrics/mAP50(B)', 0)),
                    "map50_improvement_percent": ((last_epoch.get('metrics/mAP50(B)', 0) - first_epoch.get('metrics/mAP50(B)', 0)) / max(first_epoch.get('metrics/mAP50(B)', 0.001), 0.001)) * 100,
                    "initial_train_loss": float(first_epoch.get('train/box_loss', 0)) + float(first_epoch.get('train/cls_loss', 0)),
                    "final_train_loss": float(last_epoch.get('train/box_loss', 0)) + float(last_epoch.get('train/cls_loss', 0)),
                    "loss_reduction_percent": ((first_epoch.get('train/box_loss', 0) + first_epoch.get('train/cls_loss', 0) - last_epoch.get('train/box_loss', 0) - last_epoch.get('train/cls_loss', 0)) / max(first_epoch.get('train/box_loss', 0) + first_epoch.get('train/cls_loss', 0), 0.001)) * 100
                },
                "performance_benchmarks": {
                    "excellent_map50_threshold": 0.7,
                    "good_map50_threshold": 0.5,
                    "excellent_map50_95_threshold": 0.5,
                    "good_map50_95_threshold": 0.3,
                    "map50_rating": "Excellent" if last_epoch.get('metrics/mAP50(B)', 0) > 0.7 else "Good" if last_epoch.get('metrics/mAP50(B)', 0) > 0.5 else "Needs Improvement",
                    "map50_95_rating": "Excellent" if last_epoch.get('metrics/mAP50-95(B)', 0) > 0.5 else "Good" if last_epoch.get('metrics/mAP50-95(B)', 0) > 0.3 else "Needs Improvement"
                }
            }

            # Add detailed epoch data for key milestones
            milestone_epochs = [0]  # First epoch
            milestone_epochs.extend(range(9, len(df), 10))  # Every 10th epoch
            if best_epoch_idx not in milestone_epochs and best_epoch_idx != -1:
                milestone_epochs.append(best_epoch_idx)  # Best epoch
            if len(df) - 1 not in milestone_epochs:
                milestone_epochs.append(len(df) - 1)  # Last epoch

            milestone_epochs = sorted(set(milestone_epochs))

            summary_data["training_results"]["milestone_epochs"] = []
            for i in milestone_epochs:
                row = df.iloc[i]
                milestone_data = {
                    "epoch": int(row['epoch']) + 1,
                    "is_best": i == best_epoch_idx,
                    "is_final": i == len(df) - 1,
                    "train_box_loss": float(row.get('train/box_loss', 0)),
                    "train_cls_loss": float(row.get('train/cls_loss', 0)),
                    "val_box_loss": float(row.get('val/box_loss', 0)),
                    "val_cls_loss": float(row.get('val/cls_loss', 0)),
                    "map50": float(row.get('metrics/mAP50(B)', 0)),
                    "map50_95": float(row.get('metrics/mAP50-95(B)', 0)),
                    "precision": float(row.get('metrics/precision(B)', 0)),
                    "recall": float(row.get('metrics/recall(B)', 0)),
                    "learning_rate": float(row.get('lr/pg0', 0))
                }
                summary_data["training_results"]["milestone_epochs"].append(milestone_data)

        except Exception as e:
            summary_data["training_results"] = {"error": str(e), "error_type": "results_parsing_failed"}

    # Add comprehensive recommendations based on results
    recommendations = []

    # Dataset recommendations
    if class_stats:
        classes_with_low_samples = len([c for c in class_stats.values() if c['total'] < 100])
        if classes_with_low_samples > 0:
            recommendations.append(f"Consider collecting more data for {classes_with_low_samples} classes with <100 annotations")

        imbalance_ratio = max(class_stats.values(), key=lambda x: x['total'])['total'] / max(1, min(class_stats.values(), key=lambda x: x['total'])['total'])
        if imbalance_ratio > 50:
            recommendations.append(f"Class imbalance is high ({imbalance_ratio:.1f}:1). Consider data augmentation or weighted loss functions")

    # Training recommendations
    if os.path.exists(results_csv):
        try:
            df = pd.read_csv(results_csv)
            final_map50 = df.iloc[-1].get('metrics/mAP50(B)', 0)
            if final_map50 < 0.5:
                recommendations.append("mAP@0.5 is below 0.5. Consider increasing epochs, adjusting learning rate, or collecting more data")
            elif final_map50 > 0.8:
                recommendations.append("Excellent mAP@0.5 performance! Model is ready for deployment")

            if len(df) < EPOCHS:
                recommendations.append("Training stopped early. Model may benefit from continued training or adjusted patience")
        except:
            pass

    # Hardware recommendations
    if DEVICE == 'cpu':
        recommendations.append("Training on CPU detected. Consider using GPU for faster training")

    if BATCH_SIZE < 32:
        recommendations.append("Small batch size detected. Consider increasing if GPU memory allows")

    summary_data["recommendations"] = recommendations

    # Add file structure information
    summary_data["file_structure"] = {
        "readme_file": readme_path,
        "summary_json": os.path.join(results_path, 'training_summary.json'),
        "model_weights": PATHS['trained_model'],
        "dataset_yaml": PATHS['yaml_file'],
        "results_csv": results_csv if os.path.exists(results_csv) else None,
        "expected_visualization_files": [
            "confusion_matrix.png",
            "results.png",
            "val_batch0_labels.jpg",
            "val_batch0_pred.jpg",
            "PR_curve.png",
            "F1_curve.png"
        ]
    }

    summary_path = os.path.join(results_path, 'training_summary.json')
    with open(summary_path, 'w') as f:
        json.dump(summary_data, f, indent=2, ensure_ascii=False)

    print(f"✅ Detailed summary JSON created: {summary_path}")
    print(f"📊 README includes {len(class_stats)} class statistics and comprehensive training analysis")
    print(f"🎯 {len(recommendations)} recommendations generated based on training results")

    return readme_path


create_training_readme()

In [ ]:
from IPython.display import display, HTML
import os

os.chdir('/kaggle/working')

zip_filename = f"{PROJECT_NAME}_training.zip"
folder_to_zip = "training"

print(f"📦 Creating archive: {zip_filename}")

!zip -r -q {zip_filename} {folder_to_zip}/

zip_path = f'/kaggle/working/{zip_filename}'

if os.path.exists(zip_path):
    file_size = os.path.getsize(zip_path) / (1024*1024)  # у MB
    print(f"\n✅ Archive created successfully!")
    print(f"📁 File: {zip_filename}")
    print(f"📊 Size: {file_size:.1f} MB")
    print(f"📍 Path: {zip_path}")
    
    print(f"\n📋 Archive contents:")
    !zipinfo {zip_filename} | head -20
    
    display(HTML(f'''
    <div style="background-color: #e8f5e8; padding: 15px; border-radius: 10px; margin: 10px 0;">
        <h3>📥 Download Ready</h3>
        <a href="{zip_filename}" download style="background-color: #4CAF50; color: white; padding: 10px 20px; text-decoration: none; border-radius: 5px; font-weight: bold;">
            📥 Download {zip_filename} ({file_size:.1f} MB)
        </a>
    </div>
    '''))
else:
    print("❌ Error: Archive not created!")
    print("📁 Checking working directory contents:")
    !ls -la /kaggle/working/